# 1. 유저 기반 코사인 유사도

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

ratings_df = pd.read_csv('../../data/ratings.csv')
movies_df = pd.read_csv('../../data/movies.csv')

user_item_df = ratings_df.pivot_table(index='userId', columns='movieId', values='rating')

user_item_filled_df = user_item_df.fillna(0)
user_item_filled_df


In [ ]:
user_sim_matrix = cosine_similarity(user_item_filled_df)
user_sim_df = pd.DataFrame(user_sim_matrix, index=user_item_df.index, columns=user_item_df.index).fillna(0)

# user_item_df.fillna(0, inplace=True)
user_sim_df

In [ ]:
def get_recommendations(target_user_id, num_similar_users=5, top_n=10):
    # 나랑 비슷한 상위 유저들 찾기
    similar_users = user_sim_df[target_user_id].sort_values(ascending=False).iloc[1:num_similar_users+1].index
    
    # 내가 본 영화 제외
    seen_movies = user_item_df.loc[target_user_id].dropna().index
    
    # 유사 유저들이 본 영화 중 내가 안 본 것들 추출
    sim_users_ratings = ratings_df[ratings_df['userId'].isin(similar_users)]
    recs_df = sim_users_ratings[~sim_users_ratings['movieId'].isin(seen_movies)]
    
    # 평점 기반 정렬 및 영화 정보 결합
    top_movie_ids = recs_df.groupby('movieId')['rating'].mean().sort_values(ascending=False).head(top_n).index
    return movies_df[movies_df['movieId'].isin(top_movie_ids)][['title', 'genres']]

In [ ]:
print(get_recommendations(target_user_id=1))